In [102]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [103]:
previous_application = pd.read_csv('previous_application.csv')
previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
previous_application.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL'],
      dtype='object')

In [105]:
to_drop = ['DAYS_TERMINATION','AMT_GOODS_PRICE',
'AMT_APPLICATION','RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED',
'FLAG_LAST_APPL_PER_CONTRACT', 'CODE_REJECT_REASON','NAME_CASH_LOAN_PURPOSE',
'DAYS_FIRST_DRAWING', 'DAYS_LAST_DUE_1ST_VERSION']
previous_application.drop(labels = to_drop, axis = 1, inplace = True)









In [106]:
def check_missing(df):
    df_null_percentage = df.isnull().sum() / df.shape[0] * 100
    df_null_percentage = df_null_percentage.drop(df_null_percentage[df_null_percentage == 0].index).sort_values(ascending= False).reset_index()
    df_null_percentage.columns = ["Feature", "Percentage"]
    return df_null_percentage

null_df = check_missing(previous_application)
null_df

,Feature,Percentage
0,AMT_DOWN_PAYMENT,53.636480
1,RATE_DOWN_PAYMENT,53.636480
2,NAME_TYPE_SUITE,49.119754
3,DAYS_FIRST_DUE,40.298129
4,DAYS_LAST_DUE,40.298129
5,NFLAG_INSURED_ON_APPROVAL,40.298129
6,AMT_ANNUITY,22.286665
7,CNT_PAYMENT,22.286366
8,PRODUCT_COMBINATION,0.020716
9,AMT_CREDIT,0.000060


In [107]:
# ROW > 350000 days
previous_application['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
previous_application['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)


In [108]:
check_missing(previous_application)

,Feature,Percentage
0,AMT_DOWN_PAYMENT,53.636480
1,RATE_DOWN_PAYMENT,53.636480
2,DAYS_LAST_DUE,52.944473
3,NAME_TYPE_SUITE,49.119754
4,DAYS_FIRST_DUE,42.731650
5,NFLAG_INSURED_ON_APPROVAL,40.298129
6,AMT_ANNUITY,22.286665
7,CNT_PAYMENT,22.286366
8,PRODUCT_COMBINATION,0.020716
9,AMT_CREDIT,0.000060


In [109]:

previous_application["AMT_CREDIT"].fillna(0, inplace= True)
previous_application["PRODUCT_COMBINATION"].fillna(previous_application["PRODUCT_COMBINATION"].mode()[0],inplace= True)
previous_application["NFLAG_INSURED_ON_APPROVAL"].fillna(0, inplace= True)

# Added Features

In [115]:
def extract_mean(x):
    y = x.groupby('SK_ID_CURR', as_index=False).mean().add_prefix('PREV_APPL_')
    return y


In [116]:

prev_appl = extract_mean(previous_application)
prev_appl = prev_appl.rename(columns = {'PREV_APPL_SK_ID_CURR' : 'SK_ID_CURR'})
prev_appl

/var/folders/b0/n1lz5zk51z374mpb3hyxn5n00000gp/T/ipykernel_10893/1547850365.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  y = x.groupby('SK_ID_CURR', as_index=False).mean().add_prefix('PREV_APPL_')


,SK_ID_CURR,PREV_APPL_SK_ID_PREV,PREV_APPL_AMT_ANNUITY,PREV_APPL_AMT_CREDIT,PREV_APPL_AMT_DOWN_PAYMENT,PREV_APPL_HOUR_APPR_PROCESS_START,PREV_APPL_NFLAG_LAST_APPL_IN_DAY,PREV_APPL_RATE_DOWN_PAYMENT,PREV_APPL_DAYS_DECISION,PREV_APPL_SELLERPLACE_AREA,...,PREV_APPL_DAYS_FIRST_DUE,PREV_APPL_DAYS_LAST_DUE,PREV_APPL_NFLAG_INSURED_ON_APPROVAL,PREV_APPL_PREV_APPL_APPROVED_FLAG,PREV_APPL_PREV_APPL_REFUSED_FLAG,PREV_APPL_PREV_APPL_CONSUM_LOAN_FLAG,PREV_APPL_PREV_APPL_CASH_LOAN_FLAG,PREV_APPL_PREV_APPL_REVOL_LOAN_FLAG,PREV_APPL_PREV_APPL_POS_PORTFOLIO_FLAG_COUNT,PREV_APPL_PREV_APPL_CASH_PORTFOLIO_FLAG_COUNT
0,100001,1.369693e+06,3951.000000,23787.00,2520.00,13.000000,1.0,0.104326,-1740.000,23.000,...,-1709.000000,-1619.000000,0.000000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000
1,100002,1.038818e+06,9251.775000,179055.00,0.00,9.000000,1.0,0.000000,-606.000,500.000,...,-565.000000,-25.000000,0.000000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000
2,100003,2.281150e+06,56553.990000,484191.00,3442.50,14.666667,1.0,0.050030,-1305.000,533.000,...,-1274.333333,-1054.333333,0.666667,1.00,0.00,0.666667,0.333333,0.000,0.333333,0.666667
3,100004,1.564014e+06,5357.250000,20106.00,4860.00,5.000000,1.0,0.212008,-815.000,30.000,...,-784.000000,-724.000000,0.000000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000
4,100005,2.176837e+06,4813.200000,20076.75,4464.00,10.500000,1.0,0.108964,-536.000,18.000,...,-706.000000,-466.000000,0.000000,0.50,0.00,0.500000,0.500000,0.000,0.000000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338852,456251,2.248017e+06,6605.910000,40455.00,0.00,17.000000,1.0,0.000000,-273.000,30.000,...,-210.000000,-30.000000,0.000000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000
338853,456252,1.503599e+06,10074.465000,56821.50,3456.00,10.000000,1.0,0.062443,-2497.000,190.000,...,-2466.000000,-2316.000000,1.000000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000
338854,456253,1.677221e+06,4770.405000,20625.75,4403.25,11.500000,1.0,0.214316,-2380.000,22.000,...,-2339.000000,-2219.000000,0.500000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000
338855,456254,1.904658e+06,10681.132500,134439.75,0.00,15.000000,1.0,0.000000,-299.500,1578.000,...,-269.000000,NaN,0.500000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000


In [117]:
# added features flag
previous_application['PREV_APPL_APPROVED_FLAG'] = (previous_application['NAME_CONTRACT_STATUS'] == 'Approved').astype('float')
previous_application['PREV_APPL_REFUSED_FLAG'] = (previous_application['NAME_CONTRACT_STATUS'] == 'Refused').astype('float')

previous_application['PREV_APPL_CONSUM_LOAN_FLAG'] = (previous_application['NAME_CONTRACT_TYPE'] == 'Consumer loans').astype('float')
previous_application['PREV_APPL_CASH_LOAN_FLAG'] = (previous_application['NAME_CONTRACT_TYPE'] == 'Cash loans').astype('float')
previous_application['PREV_APPL_REVOL_LOAN_FLAG'] = (previous_application['NAME_CONTRACT_TYPE'] == 'Revolving loans').astype('float')
previous_application['PREV_APPL_POS_PORTFOLIO_FLAG_COUNT'] = previous_application['NAME_PORTFOLIO'].apply(lambda x: 1 if x== "Cash" else 0)
previous_application['PREV_APPL_CASH_PORTFOLIO_FLAG_COUNT'] = previous_application['NAME_PORTFOLIO'].apply(lambda x: 1 if x== "POS" else 0)

In [119]:
previous_merge = previous_application.groupby(["SK_ID_CURR"]).agg({
                                    'PREV_APPL_APPROVED_FLAG': 'count',
                                    'PREV_APPL_REFUSED_FLAG': 'count',
                                    'PREV_APPL_CONSUM_LOAN_FLAG': 'count',
                                    'PREV_APPL_CASH_LOAN_FLAG': 'count',
                                    'PREV_APPL_REVOL_LOAN_FLAG': 'count',
                                    'PREV_APPL_POS_PORTFOLIO_FLAG_COUNT': 'count',
                                    'PREV_APPL_CASH_PORTFOLIO_FLAG_COUNT': 'count'
})

In [122]:
final_prev = previous_merge.merge(prev_appl, on = 'SK_ID_CURR', how = 'left')

In [123]:
final_prev.shape

(338857, 28)

In [124]:
prev_appl

,SK_ID_CURR,PREV_APPL_SK_ID_PREV,PREV_APPL_AMT_ANNUITY,PREV_APPL_AMT_CREDIT,PREV_APPL_AMT_DOWN_PAYMENT,PREV_APPL_HOUR_APPR_PROCESS_START,PREV_APPL_NFLAG_LAST_APPL_IN_DAY,PREV_APPL_RATE_DOWN_PAYMENT,PREV_APPL_DAYS_DECISION,PREV_APPL_SELLERPLACE_AREA,...,PREV_APPL_DAYS_FIRST_DUE,PREV_APPL_DAYS_LAST_DUE,PREV_APPL_NFLAG_INSURED_ON_APPROVAL,PREV_APPL_PREV_APPL_APPROVED_FLAG,PREV_APPL_PREV_APPL_REFUSED_FLAG,PREV_APPL_PREV_APPL_CONSUM_LOAN_FLAG,PREV_APPL_PREV_APPL_CASH_LOAN_FLAG,PREV_APPL_PREV_APPL_REVOL_LOAN_FLAG,PREV_APPL_PREV_APPL_POS_PORTFOLIO_FLAG_COUNT,PREV_APPL_PREV_APPL_CASH_PORTFOLIO_FLAG_COUNT
0,100001,1.369693e+06,3951.000000,23787.00,2520.00,13.000000,1.0,0.104326,-1740.000,23.000,...,-1709.000000,-1619.000000,0.000000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000
1,100002,1.038818e+06,9251.775000,179055.00,0.00,9.000000,1.0,0.000000,-606.000,500.000,...,-565.000000,-25.000000,0.000000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000
2,100003,2.281150e+06,56553.990000,484191.00,3442.50,14.666667,1.0,0.050030,-1305.000,533.000,...,-1274.333333,-1054.333333,0.666667,1.00,0.00,0.666667,0.333333,0.000,0.333333,0.666667
3,100004,1.564014e+06,5357.250000,20106.00,4860.00,5.000000,1.0,0.212008,-815.000,30.000,...,-784.000000,-724.000000,0.000000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000
4,100005,2.176837e+06,4813.200000,20076.75,4464.00,10.500000,1.0,0.108964,-536.000,18.000,...,-706.000000,-466.000000,0.000000,0.50,0.00,0.500000,0.500000,0.000,0.000000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338852,456251,2.248017e+06,6605.910000,40455.00,0.00,17.000000,1.0,0.000000,-273.000,30.000,...,-210.000000,-30.000000,0.000000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000
338853,456252,1.503599e+06,10074.465000,56821.50,3456.00,10.000000,1.0,0.062443,-2497.000,190.000,...,-2466.000000,-2316.000000,1.000000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000
338854,456253,1.677221e+06,4770.405000,20625.75,4403.25,11.500000,1.0,0.214316,-2380.000,22.000,...,-2339.000000,-2219.000000,0.500000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000
338855,456254,1.904658e+06,10681.132500,134439.75,0.00,15.000000,1.0,0.000000,-299.500,1578.000,...,-269.000000,NaN,0.500000,1.00,0.00,1.000000,0.000000,0.000,0.000000,1.000000


In [ ]:
final_prev.to_csv('FINAL_previous_app.csv', index = False)